# LLM でのテキスト生成

* OpenCALM-small のモデルをダウンロード → テキスト生成

注意点：
* 超軽量モデルなので、生成されるテキストは微妙。かわりにCPUでもそこそこ動作する。
* モデル自体の日本語入力は可能だが、Kubeflow UIのinput_textでの入力は英語のみ。

## transformersとPyTorchのバージョン確認とインストール

In [13]:
import transformers
transformers.__version__

'4.37.2'

In [18]:
import torch
torch.__version__

'2.2.0+cu121'

In [ ]:
!pip install transformers torch

## モデルの保存

In [15]:
# モデルの保存
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_save_path = "./trained_model"
model_name = "cyberagent/open-calm-small"

# モデルとトークナイザーのロード
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# モデルとトークナイザーの保存
tokenizer.save_pretrained(model_save_path)
model.save_pretrained(model_save_path)

## テキストの生成

In [23]:
# テキスト生成
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline #, set_seed

model_save_path = "./trained_model"

# GPUが利用可能かどうかを確認し、利用可能な場合はそのインデックスを、そうでない場合はCPUを使用する
device = 0 if torch.cuda.is_available() else -1

tokenizer = AutoTokenizer.from_pretrained(model_save_path, token=False, truncation=True)
model = AutoModelForCausalLM.from_pretrained(model_save_path, token=False)

# 生成用のパイプラインをセットアップ
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=device)

# 生成する文章のシード値を設定（オプション）
#set_seed(42)

# テキスト生成
input_text = "VMware Private AIは我々のビジネスに、"
generated_texts = generator(
    input_text,
    max_length=30,
    num_return_sequences=2,
    do_sample=True,
    temperature=0.7,
    repetition_penalty=1.2,
    top_k=50,
    pad_token_id=0
    )

for i, text in enumerate(generated_texts):
    print(f"Generated text {i+1}: {text['generated_text']}")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Generated text 1: VMware Private AIは我々のビジネスに、そして未来に向けて本当に役立つものです。
また、このプロダクトを通じて得た経験により、「Deep Lear
Generated text 2: VMware Private AIは我々のビジネスに、世界を変えるかもしれない。
「Personal Computing」とはすなわち、「コンピューティング」「クラウド・データセンター


# パイプライン YAML の作成（モデル保存のみ）

In [17]:
import kfp
from kfp import dsl
import kfp.components as comp

# コンポーネントをYAMLファイルからロード
save_op = kfp.components.load_component_from_file('save_model.yaml')

@dsl.pipeline(
    name='Save LLM Model Pipeline',
    description='Pipeline for saving an LLM model to PV.'
)

def mnist_pipeline(
        pvc_name: str = 'demo-vol-02',
        model_name: str = 'cyberagent/open-calm-small'
    ):  # PVCとModelの名前をパラメータとして追加
    
    # トレーニングタスク
    save_task = save_op(model_name=model_name)
    save_task.add_pvolumes({'/mnt/save_model': dsl.PipelineVolume(pvc=pvc_name)})

# コンパイル
kfp.compiler.Compiler().compile(mnist_pipeline, 'llm_pipeline_save-model.yaml')

# パイプライン YAML の作成（モデル保存 → テキスト生成）

In [27]:
import kfp
from kfp import dsl
import kfp.components as comp

# コンポーネントをYAMLファイルからロード
save_op = kfp.components.load_component_from_file('save_model.yaml')
gen_text_op = kfp.components.load_component_from_file('gen_text.yaml')

@dsl.pipeline(
    name='Save LLM Model and Generate Text Pipeline',
    description='Pipeline for saving an LLM model to PV, and Generate Text.'
)

def mnist_pipeline(
        model_pvc_name: str = 'demo-vol-02',
        output_pvc_name: str = 'demo-vol-03',
        model_name: str = 'cyberagent/open-calm-small',
        input_text: str = 'VMware Private AI is'
    ):  # PVCとModelの名前、入力テキストをパラメータとして追加
    
    # トレーニングタスク
    save_task = save_op(model_name=model_name)
    save_task.add_pvolumes({'/mnt/saved_model': dsl.PipelineVolume(pvc=model_pvc_name)})

    gen_text_task = gen_text_op(input_text=input_text)
    gen_text_task.add_pvolumes({
        '/mnt/saved_model': dsl.PipelineVolume(pvc=model_pvc_name),
        '/mnt/output': dsl.PipelineVolume(pvc=output_pvc_name)
    })
    gen_text_task.after(save_task)
    gen_text_task.outputs['output_text']
    
# コンパイル
kfp.compiler.Compiler().compile(mnist_pipeline, 'llm-pipeline-save-model-and-gen-text.yaml')